# How to do k-fold cross validation?
## TODOs:
### - split data into your own k folds
### - update the training and testing/validating functions to use the k folds in the correct way 
### - 

## For now, let's load in the original dataset with their given 3 fold split:

In [1]:
import os
os.chdir("..")
print(os.getcwd())

/home/wang/workspace/JupyterNoteBooksAll/fully-automated-multi-heartbeat-echocardiography-video-segmentation-and-motion-tracking


In [2]:
%config Completer.use_jedi = False

import echonet
from echonet.datasets import Echo

import torch.nn.functional as F
from torchvision.models.video import r2plus1d_18
from torch.utils.data import Dataset, DataLoader, Subset
from multiprocessing import cpu_count

from src.utils.torch_utils import TransformDataset, torch_collate
from src.transform_utils import generate_2dmotion_field
from src.loss_functions import huber_loss, convert_to_1hot, convert_to_1hot_tensor
from src.model.R2plus1D_18_MotionNet import R2plus1D_18_MotionNet  # ORIGINAL MODEL
from src.model.WIP_R2plus1D_18_MotionNet import WIP_R2plus1D_18_MotionNet # WIP MODEL
from src.echonet_dataset import EchoNetDynamicDataset
from src.clasfv_losses import deformation_motion_loss, motion_seg_loss, DiceLoss, categorical_dice
from src.train_test import train, test

import numpy as np
import matplotlib.pyplot as plt

import random
import pickle
import time

tic, toc = (time.time, time.time)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


Tensor = torch.cuda.FloatTensor

## Now we need to load in data to train a new model
### Copy and paste cells below...

### Load the indices of the subset data used for training and validating

Subset out our Train and Validation Dataset. We exclude the EchoNet videos with no clinically denoted systolic clip or ED-ES duration > 30 frames.

In [3]:
with open("fold_indexes/stanford_train_sampled_indices", "rb") as infile:
    train_mask = pickle.load(infile)
infile.close()

with open("fold_indexes/stanford_valid_sampled_indices", "rb") as infile:
    valid_mask = pickle.load(infile)
infile.close()

### Set up the training and validating dataset
work initialization function is required for generating **random** 32-frame video clip in each training epoch  
Failure to initialize the worker will cause the random 32-frame window to be the **same** for a video during every epoch.

In [4]:
batch_size = 4
num_workers = max(4, cpu_count()//2)

def worker_init_fn_valid(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    

def worker_init_fn(worker_id):
    # See here: https://pytorch.org/docs/stable/notes/randomness.html#dataloader
    # and the original post of the problem: https://github.com/pytorch/pytorch/issues/5059#issuecomment-817373837
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    

def permuter(list1, list2):
    for i1 in list1:
        for i2 in list2:
            yield (i1, i2)
            

param_trainLoader = {'collate_fn': torch_collate,
                     'batch_size': batch_size,
                     'num_workers': max(4, cpu_count()//2),
                     'worker_init_fn': worker_init_fn}

param_testLoader = {'collate_fn': torch_collate,
                    'batch_size': batch_size,
                    'shuffle': False,
                    'num_workers': max(4, cpu_count()//2),
                    'worker_init_fn': worker_init_fn}

paramLoader = {'train': param_trainLoader,
               'valid': param_testLoader,
               'test':  param_testLoader}

### Load in the dataset

In [10]:
train_dataset = EchoNetDynamicDataset(split='train', subset_indices=train_mask, period=1)
valid_dataset = EchoNetDynamicDataset(split='val', subset_indices=valid_mask, period=1)
test_dataset = EchoNetDynamicDataset(split='test', subset_indices=test_mask, period=1)
# test_dataset = EchoNetDynamicDataset(split='test', clip_length="full", raise_for_es_ed=False, period=1)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, 
                              num_workers=num_workers, 
                              shuffle=True, pin_memory=("cuda"), 
                              worker_init_fn=worker_init_fn,
                              drop_last=True)

valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, 
                              num_workers=num_workers,
                              shuffle=False, pin_memory=("cuda"),
                              worker_init_fn=worker_init_fn_valid
                             )


100%|██████████| 16/16 [00:01<00:00, 14.30it/s]


NameError: name 'test_mask' is not defined

In [ ]:
print(f'training: {len(train_dataset)}')
print(f'validation: {len(valid_dataset)}')
print(f'testing: {len(test_dataset)}')
print(f'TOTAL: {len(train_dataset) + len(valid_dataset) + len(test_dataset)}')

#### Well, that's not all 10030 videos.